In [5]:
import requests
import pandas as pd
import json
import time

url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
api_key = "17bce7d6b41147c597fde4dddd66e58c"
series_ids = [
    "CEU0000000001",
    "LNS14000000",
    "LNS11000000"
]

def fetch_bls_data(series_id):
    params = {
        "registrationKey": api_key,
        "seriesid": [series_id],
        "startyear": "2023",
        "endyear": "2024"
    }

    response = requests.post(url, data=params)
    if response.status_code == 200:
        data = response.json()
        return data['Results']['series']
    else:
        print(f"Failed to fetch data for {series_id}. Status code: {response.status_code}")
        print(response.text)
        return None


data = {}
for series_id in series_ids:
    data[series_id] = fetch_bls_data(series_id)

def parse_bls_data(raw_data):
    records = []
    for series in raw_data:
        series_id = series['seriesID']
        for item in series['data']:
            records.append({
                "series_id": series_id,
                "year": item['year'],
                "period": item['period'],
                "value": item['value'],
                "date": f"{item['year']}-{item['period'][:2]}"
            })
    return pd.DataFrame(records)

df = pd.concat([parse_bls_data(data[series_id]) for series_id in series_ids if data[series_id] is not None], ignore_index=True)

df.to_csv("bls_data.csv", index=False)